# Analysis
This notebook creates features from raw tables and visualizes the results.

## 1. Set environment
Import libraries

In [ ]:
import numpy as np
import pandas as pd
from math import floor
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from wordcloud import WordCloud, STOPWORDS
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import statsmodels.api as sm
from isodate import parse_duration
from scipy.stats import ttest_ind
from stargazer.stargazer import Stargazer

The following cell parses json files. Avoid running it again.

Counts:
- All videos
    - 1,928
- All videos with at least one comment in first 12 hours
    - 1,846
- All videos with at least one comment in English in first 12 hours
    - 1,814
- All videos with at least one comment in first 12 hours and excluding fuzzy window
    - 1516
- All videos with at least one comment in English in first 12 hours excluding fuzzy window
    - 1504
- All comments
    - 1,197,454

Read datasets

In [ ]:
# Video details table
d1 = pd.read_csv('../../dat/videoDetails.csv')

# Classified comments
# d2 = pd.read_csv('../../dat/videoFlags.csv') # Deprecated (no langid filter & no Nov-10 data)
# d2 = pd.read_csv('../../dat/videoFlagsFuzzy.csv') # Deprecated (no langid filter)
d2 = pd.read_csv('../../dat/videoFlagsFuzzyLangid.csv') # Current (has langid filter and Nov-10 data)

# Create dataframe for analysis
df = pd.merge(d1, d2, on='videoId', how='right')

Create masks for each time window

In [ ]:
# Convert publishedAt to timestamp
df['publishedAt'] = pd.to_datetime(df['publishedAt'], format='%Y-%m-%dT%H:%M:%SZ')

# Start of policy rollout
start = pd.Timestamp('2021-11-10 00:00:00')

# Time windows
hours = np.arange(12, 72+1, 12) # [12, 24, ..., 72]
windows = [f'Post{h}' for h in hours] # [Post12, Post24, ..., Post72]

# Init dict in which to store masks
donuts = {}

# Create masks for each time window
# Iterate over time windows
for h in hours:

    # Set max time of upload before treatment
    lim = start - pd.Timedelta(hours=h)

    # Current mask
    d = {
        h:(
            # Pre-treatment
            df['publishedAt'].le(lim) |
            # Post treatment
            df['publishedAt'].ge('2021-11-11')
        )
    }

    # Append masks to empty dict
    donuts = {**donuts, **d}

# Delete temporary dict
del d

## 2. Feature engineering

Turn `definition` to dummy

In [ ]:
df['definition'] = df['definition'].replace({'sd':'0','hd':'1'}).astype(int)

Create means from counters

In [ ]:
# Create per-video means
for h in hours:

    # NCRs
    df[f'ncr1Post{h}'] = df[f'post{h}CommentsNeg1'] / df[f'post{h}CommentsNum'].replace(0, 1)
    df[f'ncr2Post{h}'] = df[f'post{h}CommentsNeg2'] / df[f'post{h}CommentsNum'].replace(0, 1)
    
    # PCRs
    df[f'pcr1Post{h}'] = df[f'post{h}CommentsPos1'] / df[f'post{h}CommentsNum'].replace(0, 1)
    df[f'pcr2Post{h}'] = df[f'post{h}CommentsPos2'] / df[f'post{h}CommentsNum'].replace(0, 1)
    
    # Relative ratios
    df[f'rel1Post{h}'] = np.where(
        (df[f'post{h}CommentsNeg1'] > 0) & (df[f'post{h}CommentsPos1'] > 0),
        df[f'post{h}CommentsNeg1'] / df[f'post{h}CommentsPos1'],
        np.nan
    )
    df[f'rel2Post{h}'] = np.where(
        (df[f'post{h}CommentsNeg2'] > 0) & (df[f'post{h}CommentsPos2'] > 0),
        df[f'post{h}CommentsNeg2'] / df[f'post{h}CommentsPos2'],
        np.nan
    )

Analyze most important words in video titles to create dummy variables

In [ ]:
# All titles to single text
text = ' '.join(df['title'].str.title().tolist())

# Remove annoying strings
for string in ["'s",".","-"]:
    text = text.replace(string, '')

# All words to uppercase
text = text.upper()
# Stopwords
stopwords = set(list(STOPWORDS) + ['SAY','SAYS','S'])

# Plot wordcloud
wordcloud = WordCloud(
    background_color='white',
    max_words=25,
    stopwords=stopwords,
    max_font_size=40, 
    scale=3,
    random_state=42
).generate(text)

# Show wordcloud
fig = plt.figure(figsize=(12, 12))
plt.axis('off')
plt.imshow(wordcloud)
plt.show()

Create dummy variables by topic

In [ ]:
# Title to lowercase
df['title'] = df['title'].str.lower()

# Dictionary of keywords
topics = {
    'biden':'biden',
    'trump':'trump',
    'president':'biden|trump',
    'climate':'cop26|cop 26|climate',
    'economy':'inflation|infrastructure|bill|economy',
    'covid':'covid|covid19|covid-19|virus',
    'violence':'kill|murder|assassins| die|dead|shoot|shot'
}

# Create Indicator variables
for topic in topics.keys():
    df[topic] = np.where(df['title'].str.contains(topics[topic]), 1, 0)

Video title sentiment

In [ ]:
clf = SentimentIntensityAnalyzer()
df['toneCom'] = df['title'].apply(lambda x: clf.polarity_scores(x)['compound'])
df['tonePos'] = df['title'].apply(lambda x: clf.polarity_scores(x)['pos'])
df['toneNeg'] = df['title'].apply(lambda x: clf.polarity_scores(x)['neg'])

Translate `duration` to seconds.

In [ ]:
# YT-duration format to seconds
df['seconds'] = df['duration'].apply(lambda x: int(parse_duration(x).total_seconds()))

# log(seconds)
df['logSeconds'] = np.log(df['seconds'])

Sort data by upload date

In [ ]:
df = df.sort_values('publishedAt', ascending=True).reset_index(drop=True)

Treatment indicator

In [ ]:
df['treat'] = (df['publishedAt'] >= '2021-11-10').astype(int)

Declare running variable $R_i$ and interaction term $R_i \times T_i$
- Before: Seconds until treatment (control was positive, treatment was negative)
- Update: Seconds since treatment (control is negative, treatment is positive)

In [ ]:
# Running variable
df['r'] = (df['publishedAt'] - pd.Timestamp('2021-11-10')).dt.total_seconds()

# Interaction
df['rTreat'] = df['r'].multiply(df['treat'])

## 3. Descriptive Statistics
### Counts
Number of available videos as a function of $h$

In [ ]:
# All post%HCommentsNum columns
cols = [f'post{str(h)}CommentsNum' for h in hours]

# Merge to get videoId & post{h}CommentsNum (full list of videos!)
t = pd.merge(d1[['videoId','publishedAt']], d2[['videoId'] + cols], on='videoId', how='left')

# publishedAt to Timestamp
t['publishedAt'] = pd.to_datetime(t['publishedAt'], format='%Y-%m-%dT%H:%M:%SZ')

# Create pre & post groups
t['treat'] = t['publishedAt'] >= '2021-11-10'

# Mask each column to avoid overlap with 2021-11-10
for h, col in zip(hours, cols):
    t[col] = t['publishedAt'].le(pd.Timestamp('2021-11-10') - pd.Timedelta(hours=h)) | t['publishedAt'].ge('2021-11-11')

# Group by treatment and get counts
t = t.groupby('treat').agg(dict(zip(cols, ['sum']*7))).transpose()

# Total number of videos column
t['total'] = t.sum(axis=1)

# Format
t.index = ['h = ' + str(h) for h in windows]
t

# To latex
# print(
#    t.to_latex(
#         caption='Number of available videos before and after November 10, 2021 for different values of $h$',
#         label='tab_dat_nobs'
#     )
# )

### Balance tests

Balance table using a linear stepwise design and excluding the donut hole for $h = 12$.

$$X_i = \gamma_0 + \gamma_1 r_i + \gamma_2 T_i + \gamma_3 (r_i \times T_i) + V_i$$

In [ ]:
# Mask to avoid overlap with November 10 (h = 12)
mask = df['r'].le(-12*60**2) | df['r'].ge(24*60**2)
print(f'{mask.sum()} videos used in balance test.')

# Copy data
d = df[mask].copy()

# Create video length (minutes)
d['durationMins'] = d['seconds'].div(60)

# Order frequent-word variables by frequency
X = list(topics.keys()) + ['definition','durationMins','tonePos','toneNeg','toneCom']

# Regress each variable on r and treat
data = []
for x in X:
    m = sm.OLS.from_formula(
        data=d,
        formula=f'{x} ~ r + treat + I(r*treat)',
    ).fit(cov_type='HC0')
    data.append((m.params['treat'], m.pvalues['treat']))

# Summary table
t = pd.DataFrame(data=data, index=X, columns=['Estimated Value','p-value'])
t.index.rename('Covariate', inplace=True)
t.reset_index(inplace=True)

# Print exog that did not pass balance test
print('Variables that did not pass: \n', t.loc[t['p-value'].lt(0.1), 'Covariate'], sep='')

# View
t.round(3)

# To latex
# print(t.to_latex(caption='Regression discontinuities on observable characteristics',
#                  label='tab_dat_balance', float_format='%.3f', index=False))

## 4. Regression Analysis

Convenience functions

In [ ]:
# Stars function
def stars(pval):
    if pval <= 0.01:
        return '***'
    elif pval <= 0.05:
        return '**'
    elif pval <= 0.1:
        return '*'
    else:
        return ''

# Function to extract most relevant info from RDD
def get_info(models, dep_vars, hours):

    # Form dataframe
    t =  pd.DataFrame(
        {
            # Targets
            'y':np.repeat(dep_vars, len(hours)),
            # Time windows
            'h':list(hours)*len(dep_vars),
            # Betas
            'T':[models[i].params['treat'] for i in range(len(models))],
            # Standard errors
            'se':[models[i].bse['treat'] for i in range(len(models))],
            # p-values
            'pval':[models[i].pvalues['treat'] for i in range(len(models))],
            # Significance
            'signif':[stars(models[i].pvalues['treat']) for i in range(len(models))],
            # Mean at r=0
            'mean':[models[i].predict({'r':0, 'treat':0}).item() for i in range(len(models))],
            # R2
            'R2':[models[i].rsquared for i in range(len(models))],
            # Number of observations
            'nobs':[int(models[i].nobs) for i in range(len(models))]
        }
    )

    # Reindex table
    return t.set_index(['y', 'h'])

### Fit models without donut hole

In [ ]:
# Minimum number of comments
minComments = 0

# Empty lists to store results in
d0 = []

# Iterate over targets
for target in ['ncr1', 'ncr2', 'pcr1', 'pcr2', 'rel1', 'rel2']:

    # Iterate over Post%H windows
    for h, post in zip(hours, windows):

        # Mask (Post%HCommentsNum > 0)
        mask = df[f'{post.lower()}CommentsNum'] > 0

        # Formula for first-degree polynomial
        p1 = f'{target}{post} ~ treat + r + I(r*treat)'

        # Fit models
        m1 = sm.OLS.from_formula(formula=p1, data=df[mask]).fit(cov_type='HC0')

        # Append to list
        d0.append(m1)

In [ ]:
get_info(d0, ['ncr1', 'ncr2', 'pcr1', 'pcr2', 'rel1', 'rel2'], hours)

### Fit models with donut
`d1:` $Y_i = \beta_0 + \beta_1 r_i + \beta_2 T_i + \beta_3 (T_i \times r_i) + U_i$

`d2:` $Y_i = \beta_0 + \beta_1 r_i + \beta_2 r_i^2 + \beta_3 T_i + \beta_4 (T_i \times r_i) + \beta_5 (T_i \times r_i)^2 + V_i$

In [ ]:
# Empty lists to store results in
d1, d2 = [], []

# Iterate over targets
for target in ['ncr1', 'ncr2', 'pcr1', 'pcr2', 'rel1', 'rel2']:

    # Iterate over Post%H windows
    for h, post in zip(hours, windows):

        # Masks for `rel1` & `rel2`
        if 'rel' in target:
            mask = donuts[h] & (df[f'{target}{post}'].notna()) & (df[f'{post.lower()}CommentsNum'] > minComments)
        
        # Masks for `ncr` and `pcr`
        else:
            mask = donuts[h] & (df[f'{post.lower()}CommentsNum'] > minComments)

        # Formula for first-degree polynomial
        p1 = f'{target}{post} ~ treat + r + I(r*treat)'
        
        # Formula for second-degree polynomial
        p2 = f'{target}{post} ~ treat + r + I(r**2) + I(r*treat) + I((r*treat)**2)'

        # Fit models
        m1 = sm.OLS.from_formula(formula=p1, data=df[mask]).fit(cov_type='HC0')
        m2 = sm.OLS.from_formula(formula=p2, data=df[mask]).fit(cov_type='HC0')

        # Append to list
        d1.append(m1)
        d2.append(m2)

Summary tables for linear models

In [ ]:
d1Res = get_info(d1, ['ncr1', 'ncr2', 'pcr1', 'pcr2', 'rel1', 'rel2'], hours)
d2Res = get_info(d2, ['ncr1', 'ncr2', 'pcr1', 'pcr2', 'rel1', 'rel2'], hours)

View linear models

In [ ]:
d1Res.round(3)

View quadratic models

In [ ]:
d2Res.round(3)

### 4.4. Visualizations
Linear and quadratic RDD for $NCR(12)$

In [ ]:
# Linrear and quadratic models
l, q = d1[0], d2[0]

# Data and predictions for lines
t = pd.DataFrame({'r':df.loc[mask, 'r'],
                  'l':d1[0].fittedvalues.values,
                  'q':d2[0].fittedvalues.values})
t['treat'] = (t['r'] > 0).astype(int)

# Data for scatter
bin_length = 4
s = df.loc[mask, ['ncr1Post12','r']].copy()
s['bin'] = (s['r'].div(60 * 60) / bin_length).apply(lambda x: floor(x))
s = s.groupby('bin')['ncr1Post12'].mean().reset_index(name='mean')
s['bin'] = s['bin'].multiply(bin_length * 60 * 60)

# Initialize figure
fig, axs = plt.subplots(nrows=1, ncols=2)
fig.set_figheight(5)
fig.set_figwidth(14)

# Plot model on each axis
for i, ax in enumerate(axs):
    # Plot linear model
    if i == 0:
        series = 'l'
    else:
        series = 'q'
    # Lines
    ax.plot(t.loc[t['treat'].eq(0), 'r'], t.loc[t['treat'].eq(0), series], color='C1')
    ax.plot(t.loc[t['treat'].eq(1), 'r'], t.loc[t['treat'].eq(1), series], color='C1')
    # Scatter
    ax.scatter(s['bin'], s['mean'], color='C0', alpha=0.5)
    # Shaded regions
    ax.axvspan(xmin=0, xmax=24*60*60, color='gray', alpha=0.3)
    ax.axvspan(xmin=-12*60*60, xmax=0, color='C0', alpha=0.3)
    # X&Y axes
    ax.set_xticks(np.arange(-5*24*60*60, 6*24*60*60+1, 24 * 60 * 60))
    ax.set_xticklabels(np.arange(-5*24, 6*24+1, 24), rotation=45)
    ax.set_xlim(-5*24*60*60, 6*24*60*60)
    ax.set_ylim(0, 0.7)
    # Labels
    title = {'l':'First-degree polynomial','q':'Second-degree polynomial'}
    ax.set_title(f'{title[series]}')
    ax.set_xlabel('Hours until policy')
    ax.set_ylabel('Negative Comment Ration (h=12)')
    ax.grid(which='major', axis='x')
# Save and show
if 'google.colab' not in sys.modules:
    plt.savefig('../../fig/fig_d1d2.png', dpi=200, bbox_inches='tight')
plt.show()

All linear models

In [ ]:
# Initialize figure
fig, axs = plt.subplots(nrows=6, ncols=2)
fig.set_figheight(20)
fig.set_figwidth(14)

# Plot within each axis
for i, ax_row in enumerate(axs):
    # Left-right plots
    for j, ax in enumerate(ax_row):
        if j == 0:
            outcome = f'ncr1{windows[i]}'
            dotColor = 'blue'
            ax.set_ylabel(f'NCR(h = {hours[i]})')
            model = d1[i]
        else:
            outcome = f'ncr2{windows[i]}'
            dotColor = 'lightblue'
            ax.set_ylabel(f'sNCR(h = {hours[i]})')
            model = d1[6+i]
        # Masks
        mask = df['post12CommentsNum'].gt(0) & df[outcome].notna()
        # Line plots
        t = df.loc[mask, ['treat','r']].assign(pred = model.fittedvalues)
        ax.plot(t.loc[t['treat'].eq(0), 'r'], t.loc[t['treat'].eq(0), 'pred'], color='C1')
        ax.plot(t.loc[t['treat'].eq(1), 'r'], t.loc[t['treat'].eq(1), 'pred'], color='C1')
        # Scatter plots
        s = df[mask].copy()
        s['bin'] = (s['r'].div(4*60*60)).apply(lambda x: floor(x))
        s = s.groupby('bin')[outcome].mean().reset_index(name='mean')
        s['bin'] = s['bin'] * (4*60*60)
        ax.scatter(x=s['bin'], y=s['mean'], color='C0', alpha=0.5)
        # Shades
        ax.axvspan(xmin=0, xmax=24*60*60, color='gray', alpha=0.3)
        ax.axvspan(xmin=-hours[i]*60*60, xmax=0, color='C0', alpha=0.3)
        # Axes
        ax.set_ylim(0, 0.7)
        ax.set_xticks(np.arange(-5*24*60*60, 6*24*60*60+1, 24*60*60))
        ax.set_xticklabels(np.arange(-5*24, 6*24+1, 24))
        ax.grid(which='major', axis='x')
        if i == 0:
            ax.set_title({0:'Negative Comment Ratio',1:'Somewhat Negative Comment Ratio'}[j])
        if i == 5:
            ax.set_xlabel('Hours until policy')
# Show & save
if 'google.colab' not in sys.modules:
    plt.savefig('../../fig/fig_res_d1all.png', dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
# Initialize figure
fig, axs = plt.subplots(nrows=6, ncols=2)
fig.set_figheight(20)
fig.set_figwidth(14)

# Plot within each axis
for i, ax_row in enumerate(axs):
    # Left-right plots
    for j, ax in enumerate(ax_row):
        if j == 0:
            outcome = f'ncr1{windows[i]}'
            dotColor = 'blue'
            ax.set_ylabel(f'NCR(h = {hours[i]})')
            model = d2[i]
        else:
            outcome = f'ncr2{windows[i]}'
            dotColor = 'lightblue'
            ax.set_ylabel(f'sNCR(h = {hours[i]})')
            model = d2[6+i]
        # Masks
        mask = df['post12CommentsNum'].gt(0) & df[outcome].notna()
        # Line plots
        t = df.loc[mask, ['treat','r']].assign(pred = model.fittedvalues)
        ax.plot(t.loc[t['treat'].eq(0), 'r'], t.loc[t['treat'].eq(0), 'pred'], color='C1')
        ax.plot(t.loc[t['treat'].eq(1), 'r'], t.loc[t['treat'].eq(1), 'pred'], color='C1')
        # Scatter plots
        s = df[mask].copy()
        s['bin'] = (s['r'].div(4*60*60)).apply(lambda x: floor(x))
        s = s.groupby('bin')[outcome].mean().reset_index(name='mean')
        s['bin'] = s['bin'] * (4*60*60)
        ax.scatter(x=s['bin'], y=s['mean'], color='C0', alpha=0.5)
        # Shades
        ax.axvspan(xmin=0, xmax=24*60*60, color='gray', alpha=0.3)
        ax.axvspan(xmin=-hours[i]*60*60, xmax=0, color='C0', alpha=0.3)
        # Axes
        ax.set_ylim(0, 0.7)
        ax.set_xticks(np.arange(-5*24*60*60, 6*24*60*60+1, 24*60*60))
        ax.set_xticklabels(np.arange(-5*24, 6*24+1, 24))
        ax.grid(which='major', axis='x')
        if i == 0:
            ax.set_title({0:'Negative Comment Ratio',1:'Somewhat Negative Comment Ratio'}[j])
        if i == 5:
            ax.set_xlabel('Hours until policy')
# Show & save
if 'google.colab' not in sys.modules:
    plt.savefig('../../fig/fig_res_d2all.png', dpi=200, bbox_inches='tight')
plt.show()

Comparing linear to quadratic models

In [ ]:
# Goodness of fit
t = pd.DataFrame(
    {
        'd1R2a':[m.rsquared_adj for m in d1], 'd2R2a':[m.rsquared_adj for m in d2],
        'd1bic':[m.bic for m in d1], 'd2bic':[m.bic for m in d2],
        'd1aic':[m.aic for m in d1], 'd2aic':[m.aic for m in d2],
    }
)

t.round(4)